# UMAP example on 2D-plane data from LES simulation

Here, the UMAP dimensionality reduction algorithm will by used on a small dataset. This will be a heavily downsampled 2D plane containing instantaneous data from a 3D LES simulation of the oxidation reaction of NOx by O3 inside a turbulent jet in counterflow reactor.

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(10,8)})

In [ ]:
path = '../../data/LES/2D/2D_X_structured_subs5.csv'
data = pd.read_csv(path)
data.head()

This data was created in Paraview by converting the original unstructured plane data into a structured grid. The filters used are *Resample to Image* followed by *Extract Subset*. To do so, Paraview inserted some "ghost cells" in which all variables are zero during the first filter. In the original unstructured dataset, these cells to not exists. This is done by Paraview as an easy way to resample to the desired structured grid. These cells can be indentified by the *vtkGhostType* variable.

In [ ]:
data.groupby(['vtkGhostType']).head()

In [ ]:
data.shape

We now eliminate the ghost cells (i.e. *vtkGhostType=2*)...

In [ ]:
data.drop(data[data.vtkGhostType == 2].index, inplace=True)
data.shape

... and the *vtkGhostType* column

In [ ]:
data.drop(['vtkGhostType'], axis=1, inplace=True)
data.shape

Now we simply reset the indexes and view the data

In [ ]:
data.reset_index(drop=True, inplace=True)
data.head()

Let's visualize how the pairplots look like (for a small subset of the data as the full dataset prompts the maximum allowed size error)

In [ ]:
#sns.pairplot(data.iloc[:100])

This gives us some idea of what the data looks like by giving us all the 2D views of the data. But by reducing the dimension in a way that preserves as much of the structure of the data as possible we can get a visualisable representation of the data allowing us to "see" the data and its structure and begin to get some intuition about the data itself.

To use UMAP for this task we need to first construct a UMAP object that will do the job for us. That is as simple as instantiating the class. So let's import the umap library and do that.

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP()

Before we can do any work with the data it will help to clean up it a little. It will be helpful to convert each feature into z-scores (number of standard deviations from the mean) for comparability.

In [ ]:
scaled_data = StandardScaler().fit_transform(data)

Now we need to train our reducer, letting it learn about the manifold.
For this UMAP follows the sklearn API and has a method ``fit`` which we
pass the data we want the model to learn from. Since, at the end of the
day, we are going to want to reduced representation of the data we will
use, instead, the ``fit_transform`` method which first calls ``fit`` and
then returns the transformed data as a numpy array.

In [ ]:
embedding = reducer.fit_transform(scaled_data)
embedding.shape

The result is an array with only two feature columns. This is because, by default, UMAP
reduces down to 2D. Each row of the array is a 2-dimensional
representation of the corresponding region. Thus we can plot the
``embedding`` as a standard scatterplot and color by the target array
(since it applies to the transformed data which is in the same order as
the original).

In [ ]:
fig, ax = plt.subplots(figsize=[8, 6])
cm = plt.cm.get_cmap('inferno')
lfs = 18
tfs = 16
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=data['Phi'], vmin=0, vmax=5,
    )
plt.xticks(fontsize=tfs)
plt.yticks(fontsize=tfs)
plt.gca().set_aspect('equal', 'datalim')
cbaxes = inset_axes(ax, width="5%", height="60%", loc='upper right', borderpad=1)
cb = plt.colorbar(cax=cbaxes, orientation='vertical', ticks=[0, 1, 2, 3, 4, 5])
cbaxes.yaxis.set_ticks_position('left')
cb.ax.tick_params(labelsize=tfs)
cb.set_label(r'$\Phi$', rotation=180, size=lfs, labelpad=-50)      
cb.ax.set_yticklabels(['0', '1', '2', '3', '4', '< 5'])  

This does a useful job of capturing the structure of the data, and as
can be seen from the matrix of scatterplots this is relatively accurate.
Of course we learned at least this much just from that matrix of
scatterplots -- which we could do since we only had four different
dimensions to analyse. If we had data with a larger number of dimensions
the scatterplot matrix would quickly become unwieldy to plot, and far
harder to interpret. So moving on from the Penguin dataset, let's consider
the digits dataset.